# 這份code是在使用Jieba做斷詞並計算TF-IDF的前處理
#### Input: 
    資料們 (folder path)
#### Output:
    1.生成一個TSV檔 for Dataframe 包含:文件的id、標題、內文、全文(含標題)、內容斷詞、全文斷詞、TF-IDF(內文)以及TF-IDF(全文)
    2.生成兩個TSV檔 for bag_of_words_content、bag_of_words_whole，代表內文和全文(含標題)的bag_of_words 可做tf-idf matrix的index

In [28]:
# TEST necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setting Path

In [29]:
data_train_folder = './dataPublicComplete'
output_dataframe_path = 'dataframe_data_public.tsv'
output_bag_of_words_content_path = 'bow_content_public.tsv'
output_bag_of_words_whole_path = 'bow_whole_public.tsv'

### Preprocessing raw documents

In [30]:
import os

f_id_list = []
f_title_list = []
f_content_list = []

for filename in os.listdir(data_train_folder):
    with open(os.path.join(data_train_folder,filename), 'r', encoding="utf-8") as f:
        lines = f.read().split('\n')
        
        f_title_list.append(lines[0].strip('\n\t'))
        for line in lines[1:]:
            line.strip('\n\t')
        f_content_list.append(''.join(lines[1:]).strip('\n\t'))
        f_id_list.append(os.path.splitext(filename)[0])

In [31]:
len(f_title_list)

421

In [32]:
f_content_list[3]

'縱捲葉蟲原本在水稻第一、二期作之孕穗期至齊穗期為害最烈，但由於氣候環境異常，縱捲葉蟲為害發生的時間有提早的趨勢。94年、95年二期作水稻在插秧後不到一個月，即在成功、臺東、關山、池上等地區發現縱捲葉蟲的為害；今年二期作插秧後不久，目前已在本縣部分鄉鎮之稻田發現蟲口，尤其以越早插秧者其密度有較高的現象。因此，臺東區農業改良場提醒農友要加強防治工作，以免害蟲趁機蔓延，為害植株生長，影響水稻產量及品質。縱捲葉蟲又稱瘤野螟，產卵於葉片，淡黃色，三、四個一排；剛孵化之幼蟲為害嫩葉葉肉，二齡後即將葉尖捲成筒狀，並藏匿其中，沿葉脈取食，被害部位呈現長條白色斑痕，嚴重時會造成水稻葉片枯死。請農友巡視田間時，如發現有為害現象時即應開始施藥防治，可以選用75％歐殺松可溶性粉劑，每公頃施用0.75公斤，稀釋1,500倍；4％益滅賽寧可濕性粉劑，每公頃施用0.5公斤，稀釋2,500倍；22.5％陶斯松乳劑，每公頃施用1.1公斤，稀釋750倍；選擇其中一種來防治。並避免偏施氮肥，因成蟲有趨嫩綠性，當植株生長特別嫩綠茂盛之稻田，其產卵量會較多，為害較為嚴重。採取肥料適量的施用並與藥劑防治互相配合，可以有效遏制其發生，俾利水稻植株正常的生長。'

In [33]:
import pandas as pd
df_data_train = pd.DataFrame(list(zip(f_id_list, f_title_list, f_content_list, list(map(str.__add__, f_title_list, f_content_list)))),
               columns =['id', 'title', 'content', 'whole_text'])

df_data_train

,id,title,content,whole_text
0,1001,稻熱病進入好發季節，防檢局籲請農友加強防範,九十三年四月二十一日動植物防疫檢疫局發布之新聞稿行政院農業委員會動植物防疫檢疫局（以下簡稱防...,稻熱病進入好發季節，防檢局籲請農友加強防範九十三年四月二十一日動植物防疫檢疫局發布之新聞稿行...
1,1004,全民一起來監測紅火蟻,農委會防檢局表示，自3月份起氣溫逐漸回暖，正是紅火蟻開始活躍的季節，也是防治該害蟲的最佳時機...,全民一起來監測紅火蟻農委會防檢局表示，自3月份起氣溫逐漸回暖，正是紅火蟻開始活躍的季節，也是...
2,1006,系統測試公告,"有關96年1月29日發布之""蔥、蒜紫斑病及薊馬危害日益嚴重，台南農改場籲請農民注意及早防治""...","系統測試公告有關96年1月29日發布之""蔥、蒜紫斑病及薊馬危害日益嚴重，台南農改場籲請農民注..."
3,1008,二期作水稻生育初期，請注意縱捲葉蟲之防治,縱捲葉蟲原本在水稻第一、二期作之孕穗期至齊穗期為害最烈，但由於氣候環境異常，縱捲葉蟲為害發生...,二期作水稻生育初期，請注意縱捲葉蟲之防治縱捲葉蟲原本在水稻第一、二期作之孕穗期至齊穗期為害最...
4,1009,請農友加強水稻瘤野螟及白葉枯病防治,農委會防檢局表示，全國各地二期水稻生長陸續進入孕穗期，依據田間監測調查資料顯示，中南部局部地...,請農友加強水稻瘤野螟及白葉枯病防治農委會防檢局表示，全國各地二期水稻生長陸續進入孕穗期，依據...
...,...,...,...,...
416,991,儘速耕除綠肥作物，慎防斜紋夜蛾密度上升,一、依據新聞報導，在新竹、雲林及嘉義等地區因休耕農田種植綠肥作物（如田菁），導致斜紋夜蛾等害...,儘速耕除綠肥作物，慎防斜紋夜蛾密度上升一、依據新聞報導，在新竹、雲林及嘉義等地區因休耕農田種...
417,993,請農民在「全國滅鼠週」全力配合田間滅鼠工作,請農民在「全國滅鼠週」全力配合田間滅鼠工作行政院農業委員會動植物防疫檢疫局協同衛生、環保、國...,請農民在「全國滅鼠週」全力配合田間滅鼠工作請農民在「全國滅鼠週」全力配合田間滅鼠工作行政院農...
418,994,培育健康秧苗，預防水稻徒長病及立枯病發生,培育健康秧苗，預防水稻徒長病及立枯病發生行政院農業委員會動植物防疫檢疫局表示，目前國內各地正...,培育健康秧苗，預防水稻徒長病及立枯病發生培育健康秧苗，預防水稻徒長病及立枯病發生行政院農業委...
419,996,水稻進入分蘗盛期請農友加強防範水稻稻熱病,行政院農業委員會動植物防疫檢疫局表示，自本（九十二）年三月起全省各地水稻生長陸續進入分蘗期，...,水稻進入分蘗盛期請農友加強防範水稻稻熱病行政院農業委員會動植物防疫檢疫局表示，自本（九十二）...


### 將分好的資料用Jieba進行斷詞

In [34]:
import jieba

seg_content = []
seg_whole = []

jieba.load_userdict('./jieba/userdict.txt')
for content in df_data_train['content']:
    seg_content.append(jieba.lcut(content))
for whole_text in df_data_train['whole_text']:
    seg_whole.append(jieba.lcut(whole_text))


In [35]:
df_data_train['seg_content'] = seg_content
df_data_train['seg_whole'] = seg_whole

df_data_train

,id,title,content,whole_text,seg_content,seg_whole
0,1001,稻熱病進入好發季節，防檢局籲請農友加強防範,九十三年四月二十一日動植物防疫檢疫局發布之新聞稿行政院農業委員會動植物防疫檢疫局（以下簡稱防...,稻熱病進入好發季節，防檢局籲請農友加強防範九十三年四月二十一日動植物防疫檢疫局發布之新聞稿行...,"[九十三年, 四月, 二十一日, 動植物, 防疫, 檢疫局, 發布, 之, 新聞稿, 行政院...","[稻熱病, 進入, 好發, 季節, ，, 防, 檢局, 籲請, 農友, 加強, 防範, 九十..."
1,1004,全民一起來監測紅火蟻,農委會防檢局表示，自3月份起氣溫逐漸回暖，正是紅火蟻開始活躍的季節，也是防治該害蟲的最佳時機...,全民一起來監測紅火蟻農委會防檢局表示，自3月份起氣溫逐漸回暖，正是紅火蟻開始活躍的季節，也是...,"[農委會, 防, 檢局, 表示, ，, 自, 3, 月份, 起, 氣溫, 逐漸, 回暖, ，...","[全民, 一起, 來, 監測, 紅火蟻, 農委會, 防, 檢局, 表示, ，, 自, 3, ..."
2,1006,系統測試公告,"有關96年1月29日發布之""蔥、蒜紫斑病及薊馬危害日益嚴重，台南農改場籲請農民注意及早防治""...","系統測試公告有關96年1月29日發布之""蔥、蒜紫斑病及薊馬危害日益嚴重，台南農改場籲請農民注...","[有關, 96, 年, 1, 月, 29, 日, 發布, 之, "", 蔥, 、, 蒜, 紫斑...","[系統, 測試, 公告, 有關, 96, 年, 1, 月, 29, 日, 發布, 之, "",..."
3,1008,二期作水稻生育初期，請注意縱捲葉蟲之防治,縱捲葉蟲原本在水稻第一、二期作之孕穗期至齊穗期為害最烈，但由於氣候環境異常，縱捲葉蟲為害發生...,二期作水稻生育初期，請注意縱捲葉蟲之防治縱捲葉蟲原本在水稻第一、二期作之孕穗期至齊穗期為害最...,"[縱捲葉蟲, 原本, 在, 水稻, 第, 一, 、, 二, 期作, 之, 孕穗期, 至齊, ...","[二, 期作, 水稻, 生育, 初期, ，, 請, 注意, 縱捲葉蟲, 之, 防治, 縱捲葉..."
4,1009,請農友加強水稻瘤野螟及白葉枯病防治,農委會防檢局表示，全國各地二期水稻生長陸續進入孕穗期，依據田間監測調查資料顯示，中南部局部地...,請農友加強水稻瘤野螟及白葉枯病防治農委會防檢局表示，全國各地二期水稻生長陸續進入孕穗期，依據...,"[農委會, 防, 檢局, 表示, ，, 全國, 各地, 二期, 水稻, 生長, 陸續, 進入...","[請, 農友, 加強, 水稻, 瘤野螟, 及, 白葉枯病, 防治, 農委會, 防, 檢局, ..."
...,...,...,...,...,...,...
416,991,儘速耕除綠肥作物，慎防斜紋夜蛾密度上升,一、依據新聞報導，在新竹、雲林及嘉義等地區因休耕農田種植綠肥作物（如田菁），導致斜紋夜蛾等害...,儘速耕除綠肥作物，慎防斜紋夜蛾密度上升一、依據新聞報導，在新竹、雲林及嘉義等地區因休耕農田種...,"[一, 、, 依據, 新聞, 報導, ，, 在, 新竹, 、, 雲林, 及, 嘉義, 等, ...","[儘速, 耕除, 綠肥作物, ，, 慎防, 斜紋夜蛾, 密度, 上升, 一, 、, 依據, ..."
417,993,請農民在「全國滅鼠週」全力配合田間滅鼠工作,請農民在「全國滅鼠週」全力配合田間滅鼠工作行政院農業委員會動植物防疫檢疫局協同衛生、環保、國...,請農民在「全國滅鼠週」全力配合田間滅鼠工作請農民在「全國滅鼠週」全力配合田間滅鼠工作行政院農...,"[請, 農民, 在, 「, 全國, 滅鼠, 週, 」, 全力, 配合, 田間, 滅鼠, 工作...","[請, 農民, 在, 「, 全國, 滅鼠, 週, 」, 全力, 配合, 田間, 滅鼠, 工作..."
418,994,培育健康秧苗，預防水稻徒長病及立枯病發生,培育健康秧苗，預防水稻徒長病及立枯病發生行政院農業委員會動植物防疫檢疫局表示，目前國內各地正...,培育健康秧苗，預防水稻徒長病及立枯病發生培育健康秧苗，預防水稻徒長病及立枯病發生行政院農業委...,"[培育, 健康, 秧苗, ，, 預防, 水稻, 徒長病, 及, 立枯病, 發生, 行政院, ...","[培育, 健康, 秧苗, ，, 預防, 水稻, 徒長病, 及, 立枯病, 發生, 培育, 健..."
419,996,水稻進入分蘗盛期請農友加強防範水稻稻熱病,行政院農業委員會動植物防疫檢疫局表示，自本（九十二）年三月起全省各地水稻生長陸續進入分蘗期，...,水稻進入分蘗盛期請農友加強防範水稻稻熱病行政院農業委員會動植物防疫檢疫局表示，自本（九十二）...,"[行政院, 農業, 委員會, 動植物, 防疫, 檢疫局, 表示, ，, 自本, （, 九十二...","[水稻, 進入, 分蘗, 盛期, 請, 農友, 加強, 防範, 水稻, 稻熱病, 行政院, ..."


### Caculating TF-IDF 

In [36]:
# Preparing the format for tfidf => 'word1 <space> word2 <space> word3 ...'
contents_for_tfidf = []
for segs in df_data_train['seg_content']:
    article = ''
    for seg in segs:
        article+=seg
        article+=' '
    contents_for_tfidf.append(article)

whole_for_tfidf = []
for segs in df_data_train['seg_whole']:
    article = ''
    for seg in segs:
        article+=seg
        article+=' '
    whole_for_tfidf.append(article)


In [37]:
# Calculating term-document matrix
from sklearn.feature_extraction.text import CountVectorizer

vectoerizer1 = CountVectorizer(token_pattern='\\b\\w+\\b')
vectoerizer1.fit(contents_for_tfidf)
bag_of_words1 = vectoerizer1.get_feature_names()
term_doc_mat_content = vectoerizer1.transform(contents_for_tfidf)
print(term_doc_mat_content.toarray().shape)

vectoerizer2 = CountVectorizer(token_pattern='\\b\\w+\\b')
vectoerizer2.fit(whole_for_tfidf)
bag_of_words2 = vectoerizer2.get_feature_names()
term_doc_mat_whole = vectoerizer2.transform(whole_for_tfidf)
print(term_doc_mat_whole.toarray().shape)

C:\Users\cityrain72\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(421, 7674)
(421, 7746)


In [38]:
# Calculating TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer1 = TfidfTransformer()
tfidf_transformer1.fit(term_doc_mat_content.toarray())
tfidf_content = tfidf_transformer1.transform(term_doc_mat_content)

tfidf_transformer2 = TfidfTransformer()
tfidf_transformer2.fit(term_doc_mat_whole.toarray())
tfidf_whole = tfidf_transformer2.transform(term_doc_mat_whole)

a1 = tfidf_content.toarray()
a2 = tfidf_whole.toarray()
print(a1.shape)
print(a2.shape)



(421, 7674)
(421, 7746)


In [39]:
#Save it into dataframe 
tfidf_content_lst = [row.tolist() for row in a1]
tfidf_whole_lst = [row.tolist() for row in a2]

df_data_train['tfidf_content'] = tfidf_content_lst
df_data_train['tfidf_whole'] = tfidf_whole_lst

    

#for i in range(tfidf_content.toarray().shape[0]):
 #   df_data_train['tfidf_content'][i] = tfidf_content.toarray()[i]
  #  df_data_train['tfidf_whole'][i] = tfidf_whole.toarray()[i]
    
#df_data_train['tfidf_content'] = tfidf_content.toarray().tolist()
#df_data_train['tfidf_whole'] = tfidf_whole.toarray().tolist()


In [40]:
df_data_train

,id,title,content,whole_text,seg_content,seg_whole,tfidf_content,tfidf_whole
0,1001,稻熱病進入好發季節，防檢局籲請農友加強防範,九十三年四月二十一日動植物防疫檢疫局發布之新聞稿行政院農業委員會動植物防疫檢疫局（以下簡稱防...,稻熱病進入好發季節，防檢局籲請農友加強防範九十三年四月二十一日動植物防疫檢疫局發布之新聞稿行...,"[九十三年, 四月, 二十一日, 動植物, 防疫, 檢疫局, 發布, 之, 新聞稿, 行政院...","[稻熱病, 進入, 好發, 季節, ，, 防, 檢局, 籲請, 農友, 加強, 防範, 九十...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1004,全民一起來監測紅火蟻,農委會防檢局表示，自3月份起氣溫逐漸回暖，正是紅火蟻開始活躍的季節，也是防治該害蟲的最佳時機...,全民一起來監測紅火蟻農委會防檢局表示，自3月份起氣溫逐漸回暖，正是紅火蟻開始活躍的季節，也是...,"[農委會, 防, 檢局, 表示, ，, 自, 3, 月份, 起, 氣溫, 逐漸, 回暖, ，...","[全民, 一起, 來, 監測, 紅火蟻, 農委會, 防, 檢局, 表示, ，, 自, 3, ...","[0.0, 0.0, 0.0, 0.026067711725391532, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.02555214770201134, 0.0, 0.0,..."
2,1006,系統測試公告,"有關96年1月29日發布之""蔥、蒜紫斑病及薊馬危害日益嚴重，台南農改場籲請農民注意及早防治""...","系統測試公告有關96年1月29日發布之""蔥、蒜紫斑病及薊馬危害日益嚴重，台南農改場籲請農民注...","[有關, 96, 年, 1, 月, 29, 日, 發布, 之, "", 蔥, 、, 蒜, 紫斑...","[系統, 測試, 公告, 有關, 96, 年, 1, 月, 29, 日, 發布, 之, "",...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1008,二期作水稻生育初期，請注意縱捲葉蟲之防治,縱捲葉蟲原本在水稻第一、二期作之孕穗期至齊穗期為害最烈，但由於氣候環境異常，縱捲葉蟲為害發生...,二期作水稻生育初期，請注意縱捲葉蟲之防治縱捲葉蟲原本在水稻第一、二期作之孕穗期至齊穗期為害最...,"[縱捲葉蟲, 原本, 在, 水稻, 第, 一, 、, 二, 期作, 之, 孕穗期, 至齊, ...","[二, 期作, 水稻, 生育, 初期, ，, 請, 注意, 縱捲葉蟲, 之, 防治, 縱捲葉...","[0.0788330922052499, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0758716558335575, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1009,請農友加強水稻瘤野螟及白葉枯病防治,農委會防檢局表示，全國各地二期水稻生長陸續進入孕穗期，依據田間監測調查資料顯示，中南部局部地...,請農友加強水稻瘤野螟及白葉枯病防治農委會防檢局表示，全國各地二期水稻生長陸續進入孕穗期，依據...,"[農委會, 防, 檢局, 表示, ，, 全國, 各地, 二期, 水稻, 生長, 陸續, 進入...","[請, 農友, 加強, 水稻, 瘤野螟, 及, 白葉枯病, 防治, 農委會, 防, 檢局, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...
416,991,儘速耕除綠肥作物，慎防斜紋夜蛾密度上升,一、依據新聞報導，在新竹、雲林及嘉義等地區因休耕農田種植綠肥作物（如田菁），導致斜紋夜蛾等害...,儘速耕除綠肥作物，慎防斜紋夜蛾密度上升一、依據新聞報導，在新竹、雲林及嘉義等地區因休耕農田種...,"[一, 、, 依據, 新聞, 報導, ，, 在, 新竹, 、, 雲林, 及, 嘉義, 等, ...","[儘速, 耕除, 綠肥作物, ，, 慎防, 斜紋夜蛾, 密度, 上升, 一, 、, 依據, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
417,993,請農民在「全國滅鼠週」全力配合田間滅鼠工作,請農民在「全國滅鼠週」全力配合田間滅鼠工作行政院農業委員會動植物防疫檢疫局協同衛生、環保、國...,請農民在「全國滅鼠週」全力配合田間滅鼠工作請農民在「全國滅鼠週」全力配合田間滅鼠工作行政院農...,"[請, 農民, 在, 「, 全國, 滅鼠, 週, 」, 全力, 配合, 田間, 滅鼠, 工作...","[請, 農民, 在, 「, 全國, 滅鼠, 週, 」, 全力, 配合, 田間, 滅鼠, 工作...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
418,994,培育健康秧苗，預防水稻徒長病及立枯病發生,培育健康秧苗，預防水稻徒長病及立枯病發生行政院農業委員會動植物防疫檢疫局表示，目前國內各地正...,培育健康秧苗，預防水稻徒長病及立枯病發生培育健康秧苗，預防水稻徒長病及立枯病發生行政院農業委...,"[培育, 健康, 秧苗, ，, 預防, 水稻, 徒長病, 及, 立枯病, 發生, 行政院, ...","[培育, 健康, 秧苗, ，, 預防, 水稻, 徒長病, 及, 立枯病, 發生, 培育, 健...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
419,996,水稻進入分蘗盛期請農友加強防範水稻稻熱病,行政院農業委員會動植物防疫檢疫局表示，自本（九十二）年三月起全省各地水稻生長陸續進入分蘗期，...,水稻進入分蘗盛期請農友加強防範水稻稻熱病行政院農業委員會動植物防疫檢疫局表示，自本（九十二）...,"[行政院, 農業, 委員會, 動植物, 防疫, 檢疫局, 表示, ，, 自本, （, 九十二...","[水稻, 進入, 分蘗, 盛期, 請, 農友, 加強, 防範, 水稻, 稻熱病, 行政院, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Exporting files

In [41]:
import csv

# The dataframe
df_data_train.to_csv(output_dataframe_path, sep='\t') 

# The bag of words
with open(output_bag_of_words_content_path, 'w', newline='', encoding="utf-8") as f:
    tsv_output = csv.writer(f, delimiter='\t')
    tsv_output.writerow(bag_of_words1)
with open(output_bag_of_words_whole_path, 'w', newline='', encoding="utf-8") as f:
    tsv_output = csv.writer(f, delimiter='\t')
    tsv_output.writerow(bag_of_words2)

In [42]:
len(df_data_train['tfidf_content'][0])

7674